In [ ]:
## Gw pake subclassing yak, Biar paham aja arsitketurnya lebih dalam 

In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re as regex
from typing import Tuple, List
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, Conv2D, MaxPooling2D, BatchNormalization,Flatten
from sklearn.model_selection import train_test_split
from tensorflow.data import AUTOTUNE
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
tf.config.run_functions_eagerly(True)

#* Default Kernel
DEFAULT_KERNEL_SIZE : Tuple[int, int]= (3,3)
DEFAULT_POOL_SIZE : Tuple[int, int]= (3,3)

#* Default Filter Size  
DEFAULT_FILTER_SIZE : int = 32
DEFAULT_FILTER_SIZE2 : int = 64

#* Default Dense units
DEFAULT_UNITS1 : int = 128
DEFAULT_UNITS2 : int = 64
DEFAULT_UNITS3 : int = 512

#* Default Dropout
DROPOUT : int = 0.3

array = np.array
SIZE : int = (224,224)
DATA_PATH : str = "./data/brain_tumor/"
def preprocessImage(image : str, label : str) -> object:
	def _preprocessImage(image : str, label : str):
		image = image.numpy().decode()
		file_path = os.path.join(DATA_PATH, extract_theClass(image), image)
		image = tf.image.decode_jpeg(tf.io.read_file(file_path),channels=3)
		image = tf.image.resize(image, SIZE)
		image = image / 255
		image = tf.convert_to_tensor(image,dtype=tf.float32)
		#label = tf.convert_to_tensor(label,dtype=tf.int32)
		#image = tf.expand_dims(image, axis=0)
		#image.set_shape([ SIZE[0], SIZE[1], 1])
		return image, label
	return tf.py_function(_preprocessImage, inp=(image,label),Tout=(tf.float32, tf.float32))

def extract_theClass(image : str) -> str:	
	result = regex.findall("[a-zA-Z]+",image)[0].lower()
	if result == "no" or result == "n":
		return "no"
	elif result == "y":
		return 'yes'
	return "random"

def create_dataFrame(path_data : str) -> pd.DataFrame:
	df = pd.DataFrame(columns=["image","label"])
	list_dir = os.listdir(path_data)
	for i in list_dir:
		full_path = os.path.join(path_data, i)
		for image in os.listdir(full_path):
			temp_df = pd.DataFrame(columns=["image","label"])
			temp_df = {"image" : image, "label" : i}
			df = df.append(temp_df, ignore_index=True)
	return df

def convert_toTfDataset(datas : array,y_true: array) -> Tuple[tf.data.Dataset, tf.data.Dataset]:
	x_train,x_val,y_train,y_val = train_test_split(datas,y_true,test_size=0.2,random_state=42)
	trainDataset = tf.data.Dataset.from_tensor_slices((tf.constant(x_train,dtype=tf.string),y_train))
	valDataset = tf.data.Dataset.from_tensor_slices((tf.constant(x_val,dtype=tf.string),y_val))
	return trainDataset, valDataset


def createModel():
	model_input = Input(shape=(224,224,3))
	layer = Conv2D(filters=DEFAULT_FILTER_SIZE, kernel_size=DEFAULT_KERNEL_SIZE, activation='relu',name="conv1")(model_input)
	layer = MaxPooling2D(pool_size=DEFAULT_POOL_SIZE)(layer)
	layer = BatchNormalization()(layer)
	layer = Conv2D(filters=DEFAULT_FILTER_SIZE2, kernel_size=DEFAULT_KERNEL_SIZE, activation='relu')(layer)
	layer = MaxPooling2D(pool_size=DEFAULT_POOL_SIZE)(layer)
	layer = BatchNormalization()(layer)
	layer = Flatten()(layer)
	layer = Dense(units=DEFAULT_UNITS1, activation='relu')(layer)
	layer = Dropout(DROPOUT)(layer)
	layer = Dense(units=DEFAULT_UNITS2, activation='relu')(layer)
	layer = Dense(units=DEFAULT_UNITS3, activation='relu')(layer)
	layer = Dense(units=3, activation='softmax')(layer)
	
	model = Model(inputs=model_input, outputs=layer)
	return model

class TumorModel(Model):
	def __init__(self, num_classes, **kwargs):
		super(TumorModel, self).__init__(**kwargs)
		self.vgg16 = VGG16(weights='imagenet', include_top=False)
		self.conv1 = Conv2D(filters=DEFAULT_FILTER_SIZE, kernel_size=DEFAULT_KERNEL_SIZE, activation='relu',name="conv1",padding="SAME")
		self.conv2 = Conv2D(filters=DEFAULT_FILTER_SIZE2, kernel_size=DEFAULT_KERNEL_SIZE, activation='relu',padding="SAME",name="conv2")
		self.conv3 = Conv2D(filters=DEFAULT_FILTER_SIZE2, kernel_size=DEFAULT_KERNEL_SIZE, activation='relu',padding="VALID",name="conv3")
		self.maxPooling1 = MaxPooling2D(pool_size=DEFAULT_POOL_SIZE)
		self.batch = BatchNormalization()
		self.flatten = Flatten()
		self.dense1 = Dense(units=DEFAULT_UNITS1, activation='relu')
		self.dense2 = Dense(units=DEFAULT_UNITS2, activation='relu')
		self.dense3 = Dense(units=DEFAULT_UNITS3, activation='relu')
		self.dropout= Dropout(DROPOUT)
		self.dropout2 = Dropout(DROPOUT+0.2)
		self.outputGan = Dense(units=num_classes, activation='softmax')

	def call(self, inputs, training= True):
		self.vgg16.trainable = False
		h = self.vgg16(inputs)
		h = self.conv1(h)
		h = self.flatten(h)
		h = self.dense1(h)
		h = self.dropout(h)
		h = self.dense3(h)
		h = self.dropout2(h)
		h = self.dense2(h)
		h = self.outputGan(h)
		return h
if __name__=="__main__":
	model = TumorModel(num_classes=3)
	model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
	df = create_dataFrame(path_data="./data/brain_tumor/")
	df["label"] = df["label"].replace({"no" : 0,"yes" : 1,"random":2})
	print(df["label"].value_counts())
	#df.to_csv("./data/brain_tumor/brain_tumor.csv")
	trainDataset, valDataset = convert_toTfDataset(df["image"].values, to_categorical(df["label"].values,num_classes=3))
	trainDataset = trainDataset.map(preprocessImage)
	valDataset = valDataset.map(preprocessImage)
	trainDataset = trainDataset.batch(batch_size=32).prefetch(AUTOTUNE)
	valDataset = valDataset.batch(batch_size=32).prefetch(AUTOTUNE)
	model.fit(trainDataset, epochs=10, validation_data=valDataset)

1    155
0     98
2     24
Name: label, dtype: int64
Epoch 1/10


C:\Users\xwyzworm\anaconda3\envs\DataScience\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


1/7 [===>..........................] - ETA: 2:43 - loss: 1.0992 - accuracy: 0.3750

KeyboardInterrupt: 